In [2]:
import backtrader as bt
import datetime
%matplotlib inline

In [12]:
class TestStrategy(bt.Strategy):

    params = (
        # ('exitbars',5),
        ('maperiod',60),
    )

    def log(self,txt,dt = None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' %(dt.isoformat(),txt))
    

    def __init__(self):
        # TODO:Question1: whats inside self.datas
        self.dataclose = self.datas[0].close

        self.order = None

        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0],period = self.params.maperiod)

        bt.indicators.ExponentialMovingAverage(self.datas[0],period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0],period=25,subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi,period=10)
        bt.indicators.ATR(self.datas[0],plot=False)


    def notify_order(self,order):
        # 订单处于提交状态或者接受状态时，什么都不做
        if order.status in [order.Submitted,order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('已买入， %.2f'%order.executed.price)

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log('已卖出， %.2f'%order.executed.price)
            
            self.bar_executed = len(self)
        elif order.status in [order.Canceled,order.Margin,Order.Rejected]:
            self.log('订单失败')
        
        self.order = None
    

    def notify_trade(self,trade):
        # 一买一卖算交易
        if not trade.isclosed:
            return
        self.log('交易利润，毛利润 %.2f, 净利润 %.2f' %(trade.pnl,trade.pnlcomm))

    def next(self):
        self.log('Close %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:
            # Note: 下标是0表示今天，-1表示昨天，-2表示前天
            # if self.dataclose[0]<self.dataclose[-1]:
            #     if self.dataclose[-1]<self.dataclose[-2]:
            #         self.log('买入, %.2f' % self.dataclose[0])
            #         self.order = self.buy()

            if self.dataclose[0] > self.sma[0]:
                self.log('买入, %.2f' % self.dataclose[0])
                self.order = self.buy()

        else:
            # if len(self)>=(self.bar_executed+self.params.exitbars):
            if self.dataclose[0]<self.sma[0]:
                self.log('卖出, %.2f'%self.dataclose[0])
                self.order = self.sell()

In [10]:
cerebro = bt.Cerebro()


# 1.add data
datapath = '../data/orcl-1995-2014.txt'
mydata = bt.feeds.YahooFinanceCSVData(dataname=datapath,
                fromdate = datetime.datetime(2000,1,1),
                todate = datetime.datetime(2000,12,31),
                reverse = False)
cerebro.adddata(mydata)

# 2.set strategy
cerebro.addstrategy(TestStrategy)

# 3. set init money
cerebro.broker.set_cash(100000)

cerebro.addsizer(bt.sizers.FixedSize,stake = 10)
# Note: 原始默认每次只买一股，这里指定每次买多少股
cerebro.broker.setcommission(commission=0.001)
print(cerebro.broker.getvalue())
cerebro.run()

print(cerebro.broker.getvalue())

# cerebro.plot()


100000
2000-02-18, Close 26.05
2000-02-22, Close 26.38
2000-02-22, 买入, 26.38
2000-02-23, 已买入， 26.77
2000-02-23, Close 28.05
2000-02-24, Close 27.55
2000-02-25, Close 31.41
2000-02-28, Close 30.52
2000-02-29, Close 33.02
2000-03-01, Close 31.80
2000-03-02, Close 30.47
2000-03-03, Close 33.36
2000-03-06, Close 33.69
2000-03-07, Close 33.33
2000-03-08, Close 36.97
2000-03-09, Close 37.36
2000-03-10, Close 36.30
2000-03-13, Close 35.02
2000-03-14, Close 34.25
2000-03-15, Close 34.97
2000-03-16, Close 36.44
2000-03-17, Close 35.50
2000-03-20, Close 34.75
2000-03-21, Close 35.89
2000-03-22, Close 37.39
2000-03-23, Close 38.64
2000-03-24, Close 38.69
2000-03-27, Close 39.33
2000-03-28, Close 38.50
2000-03-29, Close 36.69
2000-03-30, Close 34.88
2000-03-30, 卖出, 34.88
2000-03-31, 已卖出， 35.66
2000-03-31, 交易利润，毛利润 88.90, 净利润 88.28
2000-03-31, Close 34.72
2000-04-03, Close 34.19
2000-04-04, Close 33.77
2000-04-05, Close 34.80
2000-04-06, Close 36.55
2000-04-06, 买入, 36.55
2000-04-07, 已买入， 37.22
2000

<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 8 Axes>]]

In [11]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 8 Axes>]]